In [ ]:
#Integrated microphone and camera killswitch

In [2]:
import tkinter as tk
import ctypes
import subprocess
import sys

# Check if the script has admin privileges
def is_admin():
    try:
        return ctypes.windll.shell32.IsUserAnAdmin()
    except:
        return False

def run_powershell_script(script):
    """Run a PowerShell script and return the output."""
    try:
        result = subprocess.run(
            ["powershell", "-Command", script],
            check=True,
            text=True,
            capture_output=True
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr.strip()}"

# Microphone Control Functions
def disable_microphone_access():
    script = """
    Set-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -Value 'Deny'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -Value 'Deny'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\Policies\\Explorer' -Name 'NoMicPrompt' -Value 1
    """
    result = run_powershell_script(script)
    update_status("Microphone access disabled.", result)

def enable_microphone_access():
    script = """
    Set-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -Value 'Allow'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -Value 'Allow'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\Policies\\Explorer' -Name 'NoMicPrompt' -Value 0
    Stop-Service -Name 'Audiosrv' -Force
    Start-Service -Name 'Audiosrv'
    """
    result = run_powershell_script(script)
    update_status("Microphone access enabled.", result)

# Camera Control Functions
def disable_camera_access():
    script = """
    Set-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -Value 'Deny'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -Value 'Deny'
    """
    result = run_powershell_script(script)
    update_status("Camera access disabled.", result)

def enable_camera_access():
    script = """
    Set-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -Value 'Allow'
    Set-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -Value 'Allow'
    """
    result = run_powershell_script(script)
    update_status("Camera access enabled.", result)

# Combined Functions
def disable_both():
    disable_microphone_access()
    disable_camera_access()

def enable_both():
    enable_microphone_access()
    enable_camera_access()

def reset_defaults():
    script = """
    Remove-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -ErrorAction SilentlyContinue
    Remove-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\microphone' -Name 'Value' -ErrorAction SilentlyContinue
    Remove-ItemProperty -Path 'HKLM:\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -ErrorAction SilentlyContinue
    Remove-ItemProperty -Path 'HKCU:\\Software\\Microsoft\\Windows\\CurrentVersion\\CapabilityAccessManager\\ConsentStore\\webcam' -Name 'Value' -ErrorAction SilentlyContinue
    """
    result = run_powershell_script(script)
    update_status("Microphone and Camera reset to defaults.", result)

# Update status label
def update_status(action, result):
    if "Error" in result:
        status_label.config(text=f"{action} failed: {result}", fg="red")
    else:
        status_label.config(text=action, fg="green")

# Request admin privileges if not already elevated
if not is_admin():
    ctypes.windll.shell32.ShellExecuteW(None, "runas", sys.executable, " ".join(sys.argv), None, 1)
    sys.exit()

# Create GUI
root = tk.Tk()
root.title("Kill Switch for Microphone and Camera")

# Buttons
tk.Button(root, text="Disable Microphone", command=disable_microphone_access, bg="red", fg="white").pack(pady=5)
tk.Button(root, text="Enable Microphone", command=enable_microphone_access, bg="green", fg="white").pack(pady=5)
tk.Button(root, text="Disable Camera", command=disable_camera_access, bg="red", fg="white").pack(pady=5)
tk.Button(root, text="Enable Camera", command=enable_camera_access, bg="green", fg="white").pack(pady=5)
tk.Button(root, text="Disable Both", command=disable_both, bg="red", fg="white").pack(pady=5)
tk.Button(root, text="Enable Both", command=enable_both, bg="green", fg="white").pack(pady=5)
tk.Button(root, text="Reset to Default", command=reset_defaults, bg="blue", fg="white").pack(pady=5)

# Status Label
status_label = tk.Label(root, text="", fg="blue")
status_label.pack(pady=10)

# Window configuration
root.geometry("300x400")
root.mainloop()

SystemExit: 